In [ ]:
data2001 = pd.read_csv('./pems07/d07_text_station_5min_2018_02_28.txt.gz',header=None, usecols=range(12))
# Assign column names based on the provided headers
column_names = [
    "Timestamp", "Station", "District", "Freeway #", 
    "Direction of Travel", "Lane Type", "Station Length", 
    "Samples", "% Observed", "Total Flow", "Avg Occupancy", "Avg Speed"
]

# Assign column names to the dataframe
data2001.columns = column_names
data2001

In [ ]:
comm = np.load('pems07_comm.npy')
comm

In [ ]:
import pandas as pd
import calendar
from datetime import date, timedelta

def read_and_process_data(file_path):
    try:
        # Read the CSV file, specifying the usecols parameter to only load the columns of interest
        data = pd.read_csv(
            file_path, 
            header=None, 
            usecols=[0, 1, 9],  # Column indexes for Timestamp, Station, and Total Flow
            names=['Timestamp', 'Station', 'Avg Flow'],  # Assigning column names
            compression='gzip'
        )

        # Convert the Timestamp column to datetime format and set it as the index
        data['Timestamp'] = pd.to_datetime(data['Timestamp'])
        data.set_index('Timestamp', inplace=True)

        # Pivot the table to get Stations as columns and Total Flow as cell values
        pivot_data = data.pivot(columns='Station', values='Avg Flow')

        return pivot_data

    except EOFError:
        print(f"Error processing file: {file_path}")
        return None

# Read the data for February 28 to get the column names (Station IDs)
# feb_28_data = read_and_process_data('./pems03/d03_text_station_5min_2018_02_28.txt.gz')
# feb_28_columns = feb_28_data.columns

# Initialize a list to hold the data for all days
all_data = []

start_date = date(2002, 6, 1)  # 开始日期
# end_date = date(2012, 10, 2)   # 结束日期
end_date = date(2024, 3, 20)   # 结束日期
current_date = start_date
all_data = []                  # 存储所有数据的列表

while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day
    file_path = f'./pems07/d07_text_station_5min_{year}_{month:02d}_{day:02d}.txt.gz'
#     if year==2012 and month==10 and day==1:
#         continue
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}, skipping...")
        current_date += timedelta(days=1)
        continue
    
    day_data = read_and_process_data(file_path)
    
    # 使用字典存储新列，以便一次性添加到数据帧
    new_columns = {}
    for col in comm:
        if col not in day_data:
            new_columns[col] = 0  # 假设使用0填充缺失的列

    # 使用 pd.concat 添加所有新列
    if new_columns:
        new_data = pd.DataFrame(new_columns, index=day_data.index)
        day_data = pd.concat([day_data, new_data], axis=1)

    # 将列重新排序以匹配2月28日的顺序
    day_data = day_data[comm]
    
    all_data.append(day_data)
    
    current_date += timedelta(days=1)

# Concatenate all daily data into a single DataFrame
combined_data = pd.concat(all_data)

# Fill any remaining missing values with 0
combined_data.fillna(0, inplace=True)

# The combined_data DataFrame now holds the merged data for January 1 to February 28
# with columns aligned to February 28 and missing values filled with 0
# Note: The code execution is commented out to prevent execution in this environment.
# combined_data.head()
combined_data.to_csv('pems12_all_common_flow.csv')

In [ ]:
import pandas as pd
pems07 = pd.read_csv('pems07_all_common_flow.csv')
pems07

data=pems07
zero_proportion = (data == 0).sum() / len(data)

# Filter out columns where the proportion of zeros is greater than 5%
columns_to_keep = zero_proportion[zero_proportion <= 0.10].index
filtered_data = data[columns_to_keep]

# import ace_tools as tools; tools.display_dataframe_to_user(name="Filtered Time Series Data", dataframe=filtered_data)
filtered_data.to_csv('pems07_all_filter10.csv',index=False)
filtered_data

In [ ]:
filtered_data.to_csv('pems07_all_filter10.csv',index=False)

In [ ]:
import pandas as pd
pems07 = pd.read_csv('pems07_all_common_flow.csv')
pems07

data=pems07
zero_proportion = (data == 0).sum() / len(data)

# Filter out columns where the proportion of zeros is greater than 5%
columns_to_keep = zero_proportion[zero_proportion <= 0.10].index
filtered_data = data[columns_to_keep]

# import ace_tools as tools; tools.display_dataframe_to_user(name="Filtered Time Series Data", dataframe=filtered_data)
filtered_data.to_csv('pems07_all_filter10.csv',index=False)
filtered_data

In [1]:
import pandas as pd
pems07 = pd.read_csv('pems07_all_filter10.csv')
pems07

,date,715898,715918,715920,715929,715930,715933,715938,715944,715947,...,765476,765477,765486,765501,765506,765508,765591,765608,765616,765618
0,2002-06-01 00:00:00,185.0,210.0,0.0,284.0,290.0,222.0,206.0,0.0,227.0,...,180.0,198.0,0.0,95.0,257.0,4.0,113.0,72.0,6.0,0.0
1,2002-06-01 00:05:00,175.0,171.0,0.0,286.0,260.0,219.0,194.0,0.0,234.0,...,231.0,164.0,0.0,91.0,237.0,2.0,156.0,69.0,10.0,0.0
2,2002-06-01 00:10:00,181.0,233.0,0.0,253.0,289.0,205.0,173.0,0.0,204.0,...,214.0,147.0,0.0,133.0,228.0,4.0,133.0,69.0,6.0,0.0
3,2002-06-01 00:15:00,171.0,195.0,0.0,279.0,294.0,202.0,167.0,0.0,196.0,...,204.0,205.0,0.0,88.0,207.0,3.0,122.0,73.0,8.0,0.0
4,2002-06-01 00:20:00,90.0,210.0,0.0,340.0,290.0,190.0,70.0,0.0,160.0,...,170.0,180.0,0.0,50.0,230.0,10.0,100.0,40.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2291275,2024-03-20 23:35:00,271.0,183.0,271.0,204.0,259.0,231.0,242.0,331.0,183.0,...,183.0,181.0,187.0,94.0,183.0,24.0,183.0,24.0,24.0,24.0
2291276,2024-03-20 23:40:00,264.0,181.0,264.0,204.0,213.0,188.0,195.0,259.0,181.0,...,181.0,213.0,172.0,105.0,182.0,23.0,182.0,23.0,23.0,23.0
2291277,2024-03-20 23:45:00,261.0,174.0,261.0,190.0,252.0,225.0,218.0,243.0,174.0,...,174.0,188.0,195.0,79.0,174.0,22.0,174.0,22.0,22.0,22.0
2291278,2024-03-20 23:50:00,248.0,165.0,247.0,194.0,224.0,198.0,181.0,220.0,165.0,...,165.0,188.0,157.0,105.0,165.0,21.0,165.0,21.0,21.0,21.0


In [2]:


# Convert the 'date' column to datetime
pems07['date'] = pd.to_datetime(pems07['date'])

# Set the 'date' column as the index
pems07.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems07_hourly = pems07.resample('H').sum()
pems07_hourly = pems07_hourly.astype('float32')

# Reset the index if you want the 'date' column back
pems07_hourly.reset_index(inplace=True)
pems07_hourly.to_csv('pems07_h.csv',index=False)



In [3]:

# Convert the 'date' column to datetime
pems07 = pd.read_csv('pems07_all_filter10.csv')
pems07['date'] = pd.to_datetime(pems07['date'])

# Set the 'date' column as the index
pems07.set_index('date', inplace=True)

# Resample to hourly data and aggregate using sum
pems07_daily = pems07.resample('D').sum()
pems07_daily = pems07_daily.astype('float32')

# Reset the index if you want the 'date' column back
pems07_daily.reset_index(inplace=True)
pems07_daily.to_csv('pems07_d.csv',index=False)